# Optimizing Model Parameters
- Now that we have a model and data it’s time to train, validate and test our model by optimizing its parameters on our data.
- Training a model is an iterative process; in each iteration the model makes a guess about the output, calculates the error in its guess (loss), collects the derivatives of the error with respect to its parameters, and **optimizes** these parameters using gradient descent.

In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

100%|██████████| 26421880/26421880 [00:02<00:00, 11339197.03it/s]


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 404098.88it/s]


Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 6194704.84it/s]


Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 16247010.53it/s]


Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw



# Hyperparameters
- Hyperparameters are adjustable parameters that let us control the model optimization process.
- Different hyperparameter values can impact model training and convergence rates.
- We define the following hyperparameters for training:
  - **Number of epochs:** The number of times to iterate over the dataset.
  - **Batch Size:** The number of data samples propagated through the network before the parameters are updated.
  - **Learning rate:** How much update we should apply to the model parameters at each batch/epoch. Smaller values yield slow learning speed, while large values may result in unpredictable behaviour during training.

In [ ]:
learning_rate = 1e-3
batch_size = 64
epochs = 5

# Optimization Loop
- Once we set our hyperparameters, we can then train and optimize our model with an optimization loop. Each iteration of the optimization loop is called an **epoch**.
- Each epoch consists of two main parts:
  - **The Train Loop** - iterate over the training dataset and try to converge to optimal parameters.
  - **The Validation/Test Loop** - iterate over the test dataset to check if model performance is improving.

# Loss Function
- Loss function measures the degree of dissimilarity of obtained result to the target value, and it is the loss function that we want to minimize during training. - To calculate the loss we make a prediction using the inputs of our given data sample and compare it against the true data label value.
- Common loss functions include `nn.MSELoss (Mean Square Error)` for regression tasks, and `nn.NLLLoss (Negative Log Likelihood)` for classification. `nn.CrossEntropyLoss` combines `nn.LogSoftmax` and `nn.NLLLoss`.
- We pass our model’s output logits to nn.CrossEntropyLoss, which will normalize the logits and compute the prediction error.

In [ ]:
# Initialize the loss function
loss_fn = nn.CrossEntropyLoss()

# Optimizer
- Optimization is the process of adjusting model parameters to reduce model error in each training step.
- **Optimization algorithms** define how this process is performed (in this example we use Stochastic Gradient Descent).
- All optimization logic is encapsulated in the `optimizer` object.
- Optimizers available in PyTorch includes: `SGD`, `ADAM`, and `RMSProp`.
- We initialize the optimizer by registering the model’s parameters that need to be trained, and passing in the learning rate hyperparameter.


In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

Inside the training loop, optimization happens in three steps:
- Call `optimizer.zero_grad()` to reset the gradients of model parameters. Gradients by default add up; to prevent double-counting, we explicitly zero them at each iteration.
- Backpropagate the prediction loss with a call to `loss.backward()`. PyTorch deposits the gradients of the loss w.r.t. each parameter.
- Once we have our gradients, we call `optimizer.step()` to adjust the parameters by the gradients collected in the backward pass.

# Full Implementation
We define `train_loop` that loops over our optimization code, and `test_loop` that evaluates the model's performance against our test data.

In [ ]:
def train_loop(dataloader, model, loss_fn, optimizer):
  size = len(dataloader.dataset)
  # Set the model to training mode - important for batch normalization and dropout layers
  # Unnecessary in this situation but added for best practices
  model.train()
  for batch, (X, y) in enumerate(dataloader):
    # Compute prediction and loss
    pred = model(X)
    loss = loss_fn(pred, y)

    # Backpropagation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch%100 == 0:
      loss, current = loss.item(), batch * batch_size + len(X)
      print(f"loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader, model, loss_fn):
    # Set the model to evaluation mode - important for batch normalization and dropout layers
    # Unnecessary in this situation but added for best practices
    model.eval()
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    # Evaluating the model with torch.no_grad() ensures that no gradients are computed during test mode
    # also serves to avoid unnecessary gradient computations and memory usage for tensors with requires_grad=True
    with torch.no_grad():
      for X, y in dataloader:
        pred = model(X)
        test_loss += loss_fn(pred, y).item()
        correct += (pred.argmax(dim=1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")


In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = learning_rate)
epochs = 10

for t in range(epochs):
  print(f"Epoch {t+1}\n -----------------------")
  train_loop(train_dataloader, model, loss_fn, optimizer)
  test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
 -----------------------
loss: 2.297175 [   64/60000]
loss: 2.290338 [ 6464/60000]
loss: 2.268400 [12864/60000]
loss: 2.269954 [19264/60000]
loss: 2.247679 [25664/60000]
loss: 2.221757 [32064/60000]
loss: 2.232344 [38464/60000]
loss: 2.196507 [44864/60000]
loss: 2.190325 [51264/60000]
loss: 2.158698 [57664/60000]
Test Error: 
 Accuracy: 50.5%, Avg loss: 2.154517 

Epoch 2
 -----------------------
loss: 2.157152 [   64/60000]
loss: 2.152339 [ 6464/60000]
loss: 2.090851 [12864/60000]
loss: 2.113436 [19264/60000]
loss: 2.067412 [25664/60000]
loss: 2.003962 [32064/60000]
loss: 2.030914 [38464/60000]
loss: 1.951374 [44864/60000]
loss: 1.952869 [51264/60000]
loss: 1.881200 [57664/60000]
Test Error: 
 Accuracy: 60.6%, Avg loss: 1.878923 

Epoch 3
 -----------------------
loss: 1.904539 [   64/60000]
loss: 1.880034 [ 6464/60000]
loss: 1.757731 [12864/60000]
loss: 1.804962 [19264/60000]
loss: 1.705439 [25664/60000]
loss: 1.651705 [32064/60000]
loss: 1.665070 [38464/60000]
loss: 1.571437